In [ ]:
from dataclasses import dataclass, field
import hashlib
import itertools
from typing import List, Dict, Any, Tuple, Optional
import json
import os

import pandas as pd
import numpy as np

# import cosine similarity
from sklearn.metrics.pairwise import cosine_similarity

from rag_tools import validate_recommendations

In [133]:
with open("/openai/api_key.json") as f:
    config = json.load(f)


os.environ["OPENAI_API_KEY"] = config["key"]

NameError: name 'os' is not defined

In [ ]:
test_vocab = "word_list5.pkl"

# load the vocabulary
df = pd.read_pickle(test_vocab)
df.shape

(121, 3)

In [ ]:
@dataclass
class ConnectionGroup:
    group_metric: float = field(default=0.0, metadata={"help": "Average cosine similarity of all combinations of words in the group"})
    root_word: str = field(default="", metadata={"help": "Root word of the group"})
    candidate_pairs: list = field(default_factory=list, metadata={"help": "List of candidate word with definition"})
    group_id: Optional[str] = field(default=None, metadata={"help": "Checksum identifer for the group"})

    def add_entry(self, word, connection):
        if len(self.candidate_pairs) < 4:
            self.candidate_pairs.append((word, connection))
            if len(self.candidate_pairs) == 4:
                self.group_id = self._compute_group_id()
        else:
            raise ValueError("Group is full, cannot add more entries")          

    def get_candidate_words(self):
        sorted_pairs = sorted(self.candidate_pairs, key=lambda x:x[0])
        return [x[0] for x in sorted_pairs]
    
    def get_candidate_connections(self):
        sorted_pairs = sorted(self.candidate_pairs, key=lambda x:x[0])

        # strip the part of speech tag at the beginning of the connection, which looks like "noun:" or "verb:" etc.
        # find the first colon and take the substring after it
        stripped_connections = [x[1].split(':', 1)[1].strip() if ':' in x[1] else x[1] for x in sorted_pairs]

        return stripped_connections
    
    def _compute_group_id(self):
        return hashlib.md5("".join(self.get_candidate_words()).encode()).hexdigest()

    def __repr__(self):
        return_string = f"group metric: {self.group_metric}, "
        return_string += f"root word: {self.root_word}, group id: {self.group_id}\n"
        return_string += f"candidate group: {self.get_candidate_words()}\n"
        for connection in self.get_candidate_connections():
            return_string += f"\t{connection}\n"

        return return_string
    
    # method to determine if the group is equal to another group
    def __eq__(self, other):
        return set(self.get_candidate_words()) == set(other.get_candidate_words())

In [ ]:
df.head(15)

,word,definition,embedding
0,uphold,"verb: to maintain or support something, such a...","[0.05433855205774307, 0.028151938691735268, 0...."
1,uphold,verb: to lift or hold something in an elevated...,"[0.006949818227440119, -0.0720582902431488, -0..."
2,uphold,verb: to defend or stand up for a principle or...,"[0.0063194772228598595, -0.022249704226851463,..."
3,discard,verb: To throw something away because it is no...,"[0.023533722385764122, 0.005563678685575724, -..."
4,discard,"verb: To get rid of a card in a game, usually ...","[-6.281930836848915e-05, -0.023267202079296112..."
5,discard,noun: The act of throwing something away.,"[0.013182280585169792, 0.010861128568649292, -..."
6,discard,noun: A card that has been thrown away in a ca...,"[0.0007180095999501646, -0.007413978222757578,..."
7,honor,noun: Respect and esteem shown to someone or s...,"[0.05223957821726799, -0.04617014899849892, -0..."
8,honor,noun: A privilege or mark of distinction.,"[0.048169057816267014, -0.009565112181007862, ..."
9,honor,"noun: A code of integrity and ethics, often as...","[0.022865494713187218, -0.0090312659740448, 0...."


In [ ]:
def get_candidate_words(df: pd.DataFrame) -> list:
    """
    Generate a list of candidate word groups based on cosine similarity of their embeddings.

    Args:
        df (pd.DataFrame): DataFrame containing words and their corresponding embeddings. Dataframe should have two columns: 'word', 'definition' and 'embedding', in that order.

    Returns:
        list: A list of unique candidate word groups sorted by their group metric in descending order.
    """

    candidate_list = []

    # create cosine similarity matrix for all pairs of the vectors
    cosine_similarities = cosine_similarity(df['embedding'].tolist())
    print(cosine_similarities.shape)

    # for each row in the cosine similarity matrix, sort by the cosine similarity
    sorted_cosine_similarites = np.argsort(cosine_similarities, axis=1)
    print(sorted_cosine_similarites.shape)

    # group of words that are most similar to each other
    for r in range(df.shape[0]):

        # get the top 3 closest words that are not the same as the current word and are not already connected
        connected_words = set()
        top3 = []
        for i in range(sorted_cosine_similarites.shape[1]-2, 0, -1):
            c = sorted_cosine_similarites[r, i]
            
            # make sure the word is not already connected and not the current word
            if df.iloc[c, 0] not in connected_words and df.iloc[c, 0] != df.iloc[r, 0]:
                connected_words.add(df.iloc[c, 0])
                top3.append(c)
            if len(connected_words) == 3:
                break   

        # create candidate group for the current word and the top 3 closest words
        if df.iloc[r, 0] not in connected_words and len(connected_words) == 3:
            candidate_group = ConnectionGroup()
            candidate_group.group_metric = cosine_similarities[r, top3].mean()
            candidate_group.root_word = df.iloc[r, 0]
            candidate_group.add_entry(df.iloc[r, 0], df.iloc[r, 1])
            
            for c in top3:
                candidate_group.add_entry(df.iloc[c, 0], df.iloc[c, 1])

            combinations = list(itertools.combinations([r] + top3, 2))
            candidate_group.group_metric= np.array([cosine_similarities[r, c] for r,c in combinations]).mean()


            candidate_list.append(candidate_group)

    # sort the candidate list by the group metric in descending order
    candidate_list.sort(key=lambda x: x.group_metric, reverse=True)

    # remove duplicate groups
    found_groups = set()
    unique_candidate_list = []
    for candidate in candidate_list:
        if candidate.group_id not in found_groups:
            unique_candidate_list.append(candidate)
            found_groups.add(candidate.group_id)

    return unique_candidate_list

In [134]:
candidate_list = get_candidate_words(df)
print(len(candidate_list))
list_to_validate = "\n".join([str(x) for x in candidate_list[:10]])
recommendation_list = validate_recommendations(list_to_validate)
print(recommendation_list)

(121, 121)
(121, 121)
86


OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [125]:
print(list_to_validate)

group metric: 0.6683597911897255, root word: blanket, group id: 7f7f5db443336ea81c19f94b4a07d8d8
candidate group: ['blanket', 'sham', 'sheet', 'throw']
	A large piece of fabric used to cover and keep warm, typically on a bed.
	A decorative pillow covering or case.
	A large rectangular piece of fabric used for covering a bed.
	a light blanket or cover used for warmth or decoration

group metric: 0.6067497164833743, root word: keep, group id: 02f440f2aa99d7c860db64d79479f106
candidate group: ['fulfill', 'honor', 'keep', 'uphold']
	to carry out a task or duty as required or expected
	To fulfill or keep an agreement or commitment.
	to fulfill or adhere to a commitment or promise
	to maintain or support something, such as a decision or a law

group metric: 0.580787881810623, root word: pass, group id: 5df52bf83b22266cd2b9516cbf70763e
candidate group: ['fulfill', 'honor', 'keep', 'pass']
	to satisfy a requirement, condition, or need
	To fulfill or keep an agreement or commitment.
	to fulfill

In [101]:
#  remove from df_combined the rows where the word is in candidate_words
df = df[~df['word'].isin(candidate_list[0].get_candidate_words())]
df.shape

(29, 3)

In [102]:
len(df.word.unique()),df.word.unique()

(4, array(['discard', 'state', 'justice', 'draw'], dtype=object))

In [103]:
candidate_list = get_candidate_words(df)
print(len(candidate_list))
for candidate in candidate_list[:10]:
    print(candidate)

(29, 29)
(29, 29)
1
group metric: 0.3307956064031187, root word: draw, group id: 1815c76bfe1f68bd0508575e7c044575
candidate group: ['discard', 'draw', 'justice', 'state']
	The act of throwing something away.
	an act of selecting a winner or winners in a lottery or similar game of chance
	The legal system or the act of applying laws
	To specify or set forth, e.g., 'The contract states the terms of the agreement.'



In [104]:
#  remove from df_combined the rows where the word is in candidate_words
df = df[~df['word'].isin(candidate_list[0].get_candidate_words())]
df.shape

(0, 3)

len(df.word.unique()),df.word.unique()

candidate_list = get_candidate_words(df)
print(len(candidate_list))
for candidate in candidate_list[:20]:
    print(candidate)

#  remove from df_combined the rows where the word is in candidate_words
df = df[~df['word'].isin(candidate_list[0].get_candidate_words())]
df.shape

len(df.word.unique()),df.word.unique()